In [ ]:
from keras.applications.vgg16 import VGG16

In [ ]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Conv2D, Flatten,MaxPooling2D, Dropout
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()

In [ ]:
x_train.shape

(50000, 32, 32, 3)

In [ ]:
x_test.shape

(10000, 32, 32, 3)

In [ ]:
x_train = x_train/255

In [ ]:
x_test = x_test/255

In [ ]:
x_train[:1].shape

(1, 32, 32, 3)

In [ ]:
# datagen = ImageDataGenerator(
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest'
# )

# # Fit the ImageDataGenerator on your training data
# datagen.fit(x_train)

# # Create a generator that will apply data augmentation on the fly during training
# augmented_generator = datagen.flow(x_train, y_train, batch_size=32)

In [ ]:
from skimage.transform import resize
new_shape = (150, 150, 3)
x_train_resized = [resize(img, new_shape) for img in x_train]
x_test_resized = [resize(img, new_shape) for img in x_test]

In [ ]:
conv_base = VGG16(weights = 'imagenet', include_top = False, input_shape = (150, 150,3))

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [ ]:
conv_base.trainable = False

In [ ]:
model = Sequential()

model.add(conv_base)

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(100, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 128)               1048704   
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 100)               6500      
                                                                 
Total params: 15778148 (60.19 MB)
Trainable params: 1063460 (4.06 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [ ]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping

In [ ]:
callbacks = EarlyStopping(monitor = 'val_loss', patience = 3, verbose = 1, mode = 'auto')

In [ ]:
history = model.fit(x_train_resized, y_train, epochs = 100, batch_size = 32, verbose =1 , validation_data = (x_test_resized, y_test), callbacks = [callbacks])